In [4]:
from utils import getLoaderDataset

N = 256
B = 1

loader, dataset = getLoaderDataset(N, B, "./datasets/shakespear_corpus.txt", 10000)

In [5]:
from model import GPT, train_model

L = 6
d = 384
d_ff = 4 * d
h = 6
V = dataset.get_vocab_size()

model = GPT(B, L, d, d_ff, N, h, V)

In [6]:
model, losses = train_model(model, loader, dataset)

torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 256]) torch.Size([1

In [4]:
from utils import generate

new_tokens = generate(model, dataset.encode("First"), 50)
print(dataset.decode(new_tokens))

/home/gordon/Documents/edu/gordon_ms/Sem3/DeepLearning/Shakespeak/utils.py:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_c = torch.tensor(char_id).reshape(shape=(1, 1))


First EdesdlAener. cagonK! cayou chuth, Wchgongout, one
